## สำรวจการจัดกลุ่ม K-Means ด้วย R และหลักการข้อมูลแบบ Tidy

### [**แบบทดสอบก่อนเรียน**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

ในบทเรียนนี้ คุณจะได้เรียนรู้วิธีสร้างกลุ่มโดยใช้แพ็กเกจ Tidymodels และแพ็กเกจอื่นๆ ในระบบนิเวศของ R (เราจะเรียกพวกมันว่าเพื่อน 🧑‍🤝‍🧑) รวมถึงชุดข้อมูลเพลงไนจีเรียที่คุณนำเข้าไว้ก่อนหน้านี้ เราจะครอบคลุมพื้นฐานของ K-Means สำหรับการจัดกลุ่ม โปรดจำไว้ว่า ตามที่คุณได้เรียนรู้ในบทเรียนก่อนหน้า มีหลายวิธีในการทำงานกับการจัดกลุ่ม และวิธีที่คุณใช้ขึ้นอยู่กับข้อมูลของคุณ เราจะลองใช้ K-Means เนื่องจากเป็นเทคนิคการจัดกลุ่มที่พบได้บ่อยที่สุด มาเริ่มกันเลย!

คำศัพท์ที่คุณจะได้เรียนรู้:

-   การให้คะแนน Silhouette

-   วิธี Elbow

-   ความเฉื่อย (Inertia)

-   ความแปรปรวน (Variance)

### **บทนำ**

[K-Means Clustering](https://wikipedia.org/wiki/K-means_clustering) เป็นวิธีที่มาจากสาขาการประมวลผลสัญญาณ ใช้ในการแบ่งและจัดกลุ่มข้อมูลออกเป็น `k clusters` โดยอิงจากความคล้ายคลึงกันของคุณลักษณะ

กลุ่มเหล่านี้สามารถแสดงผลเป็น [แผนภาพ Voronoi](https://wikipedia.org/wiki/Voronoi_diagram) ซึ่งประกอบด้วยจุด (หรือ 'seed') และพื้นที่ที่เกี่ยวข้อง

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>อินโฟกราฟิกโดย Jen Looper</figcaption>

ขั้นตอนของ K-Means clustering มีดังนี้:

1. นักวิทยาศาสตร์ข้อมูลเริ่มต้นโดยกำหนดจำนวนกลุ่มที่ต้องการสร้าง

2. จากนั้น อัลกอริทึมจะสุ่มเลือก K ข้อมูลจากชุดข้อมูลเพื่อใช้เป็นจุดศูนย์กลางเริ่มต้นสำหรับกลุ่ม (หรือที่เรียกว่า centroids)

3. ต่อมา ข้อมูลที่เหลือทั้งหมดจะถูกจัดกลุ่มไปยัง centroid ที่ใกล้ที่สุด

4. จากนั้น คำนวณค่าเฉลี่ยใหม่ของแต่ละกลุ่ม และ centroid จะถูกย้ายไปยังตำแหน่งค่าเฉลี่ยนั้น

5. เมื่อจุดศูนย์กลางถูกคำนวณใหม่แล้ว ข้อมูลทุกชิ้นจะถูกตรวจสอบอีกครั้งเพื่อดูว่ามันอาจใกล้กับกลุ่มอื่นมากกว่า ข้อมูลทั้งหมดจะถูกจัดกลุ่มใหม่โดยใช้ค่าเฉลี่ยของกลุ่มที่อัปเดต ขั้นตอนการจัดกลุ่มและการอัปเดต centroid จะถูกทำซ้ำจนกว่าการจัดกลุ่มจะหยุดเปลี่ยนแปลง (หรือเมื่อเกิดการลู่เข้า) โดยทั่วไป อัลกอริทึมจะหยุดเมื่อการเคลื่อนที่ของ centroid ในแต่ละรอบใหม่มีน้อยมาก และกลุ่มกลายเป็นคงที่

<div>

> โปรดทราบว่าเนื่องจากการสุ่มเลือกข้อมูลเริ่มต้น k ที่ใช้เป็น centroid การดำเนินการอาจให้ผลลัพธ์ที่แตกต่างกันเล็กน้อยในแต่ละครั้ง ด้วยเหตุนี้ อัลกอริทึมส่วนใหญ่จึงใช้การเริ่มต้นแบบสุ่มหลายครั้ง (*random starts*) และเลือกการวนซ้ำที่มีค่า WCSS ต่ำที่สุด ดังนั้นจึงแนะนำอย่างยิ่งให้รัน K-Means ด้วยค่าของ *nstart* หลายค่าเพื่อหลีกเลี่ยง *local optimum* ที่ไม่พึงประสงค์

</div>

แอนิเมชันสั้นๆ นี้ใช้ [งานศิลปะ](https://github.com/allisonhorst/stats-illustrations) ของ Allison Horst อธิบายกระบวนการจัดกลุ่ม:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>งานศิลปะโดย @allison_horst</figcaption>

คำถามพื้นฐานที่เกิดขึ้นในการจัดกลุ่มคือ: คุณจะรู้ได้อย่างไรว่าควรแยกข้อมูลออกเป็นกี่กลุ่ม? ข้อเสียของการใช้ K-Means คือคุณจะต้องกำหนด `k` ซึ่งก็คือจำนวน `centroids` โชคดีที่ `elbow method` ช่วยประมาณค่าที่ดีสำหรับการเริ่มต้น `k` คุณจะได้ลองใช้ในอีกสักครู่

### 

**ข้อกำหนดเบื้องต้น**

เราจะเริ่มต้นจากจุดที่เราหยุดไว้ใน [บทเรียนก่อนหน้า](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb) ซึ่งเราได้วิเคราะห์ชุดข้อมูล สร้างภาพจำนวนมาก และกรองชุดข้อมูลไปยังข้อมูลที่น่าสนใจ อย่าลืมตรวจสอบ!

เราจะต้องใช้แพ็กเกจบางตัวเพื่อดำเนินการในโมดูลนี้ คุณสามารถติดตั้งได้โดยใช้: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

หรือใช้สคริปต์ด้านล่างเพื่อตรวจสอบว่าคุณมีแพ็กเกจที่จำเป็นสำหรับโมดูลนี้หรือไม่ และติดตั้งให้ในกรณีที่ขาดหาย


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


มาลุยกันเลย!

## 1. เต้นรำกับข้อมูล: คัดเลือก 3 แนวเพลงที่ได้รับความนิยมมากที่สุด

นี่คือการทบทวนสิ่งที่เราได้ทำในบทเรียนก่อนหน้า มาลองจัดการข้อมูลกันเถอะ!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 นั่นไปได้ดีมาก!

## 2. การสำรวจข้อมูลเพิ่มเติม

ข้อมูลนี้สะอาดแค่ไหน? มาลองตรวจสอบค่าผิดปกติด้วยการใช้กล่องแสดงค่ากัน เราจะเน้นไปที่คอลัมน์ตัวเลขที่มีค่าผิดปกติน้อยกว่า (แม้ว่าคุณจะสามารถลบค่าผิดปกติออกได้) กล่องแสดงค่าจะช่วยแสดงช่วงของข้อมูลและช่วยเลือกว่าคอลัมน์ใดควรนำมาใช้ อย่างไรก็ตาม กล่องแสดงค่าไม่ได้แสดงค่าความแปรปรวน ซึ่งเป็นองค์ประกอบสำคัญของข้อมูลที่สามารถจัดกลุ่มได้ดี กรุณาดู [การอภิปรายนี้](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) เพื่ออ่านเพิ่มเติม

[กล่องแสดงค่า](https://en.wikipedia.org/wiki/Box_plot) ถูกใช้เพื่อแสดงการกระจายของข้อมูล `ตัวเลข` ในรูปแบบกราฟิก ดังนั้นเรามาเริ่มต้นด้วยการ *เลือก* คอลัมน์ตัวเลขทั้งหมดควบคู่ไปกับแนวเพลงยอดนิยมกัน


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


ดูว่า `where` ใน selection helper ทำให้เรื่องนี้ง่ายขึ้นแค่ไหน 💁? ลองสำรวจฟังก์ชันอื่นๆ ได้ [ที่นี่](https://tidyselect.r-lib.org/)  

เนื่องจากเราจะสร้าง boxplot สำหรับแต่ละคุณสมบัติที่เป็นตัวเลข และเราต้องการหลีกเลี่ยงการใช้ loops ลองปรับรูปแบบข้อมูลของเราให้เป็น *longer* format ซึ่งจะช่วยให้เราใช้ `facets` ได้ - subplots ที่แสดงข้อมูลแต่ละชุดแยกกัน


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


ถึงเวลาสนุกกับ `ggplots` กันแล้ว! แล้วเราจะใช้ `geom` อะไรดี?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


ง่ายมาก!

ตอนนี้เราสามารถเห็นได้ว่าข้อมูลนี้ค่อนข้างมีความแปรปรวน: เมื่อดูแต่ละคอลัมน์ในรูปแบบกล่องแสดงค่ากระจาย คุณจะเห็นค่าผิดปกติ คุณอาจจะไล่ดูชุดข้อมูลและลบค่าผิดปกติเหล่านี้ออกไป แต่การทำเช่นนั้นจะทำให้ข้อมูลเหลือน้อยลงมาก

สำหรับตอนนี้ เรามาเลือกคอลัมน์ที่เราจะใช้สำหรับการฝึกการจัดกลุ่มกันดีกว่า เราจะเลือกคอลัมน์ตัวเลขที่มีช่วงค่าคล้ายกัน เราสามารถเข้ารหัส `artist_top_genre` เป็นตัวเลขได้ แต่ตอนนี้เราจะตัดมันออกไปก่อน


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. การคำนวณ k-means clustering ใน R

เราสามารถคำนวณ k-means ใน R ได้โดยใช้ฟังก์ชัน `kmeans` ที่มีอยู่แล้ว ดูเพิ่มเติมได้ที่ `help("kmeans()")` ฟังก์ชัน `kmeans()` รับข้อมูลในรูปแบบ data frame ที่มีคอลัมน์เป็นตัวเลขทั้งหมดเป็นอาร์กิวเมนต์หลัก

ขั้นตอนแรกในการใช้ k-means clustering คือการกำหนดจำนวนคลัสเตอร์ (k) ที่จะสร้างขึ้นในผลลัพธ์สุดท้าย เราทราบว่ามี 3 ประเภทของเพลงที่เราแบ่งออกมาจากชุดข้อมูล ดังนั้นลองใช้ค่า k เป็น 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


วัตถุ kmeans มีข้อมูลหลายส่วนที่อธิบายได้ดีใน `help("kmeans()")` สำหรับตอนนี้ เรามาเน้นที่บางส่วนกัน เราเห็นว่าข้อมูลถูกจัดกลุ่มเป็น 3 กลุ่ม โดยมีขนาด 65, 110, 111 ผลลัพธ์ยังมีจุดศูนย์กลางของกลุ่ม (ค่าเฉลี่ย) สำหรับ 3 กลุ่มใน 5 ตัวแปร

เวกเตอร์การจัดกลุ่มคือการกำหนดกลุ่มสำหรับแต่ละการสังเกตการณ์ ลองใช้ฟังก์ชัน `augment` เพื่อเพิ่มการกำหนดกลุ่มลงในชุดข้อมูลเดิม


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


เยี่ยมเลย! ตอนนี้เราได้แบ่งชุดข้อมูลของเราออกเป็น 3 กลุ่มแล้ว ทีนี้คำถามคือ การจัดกลุ่มของเราดีแค่ไหน 🤷? มาดูที่ `Silhouette score` กันเถอะ

### **Silhouette score**

[การวิเคราะห์ Silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) สามารถใช้เพื่อศึกษาระยะห่างระหว่างกลุ่มที่ได้จากการจัดกลุ่ม คะแนนนี้มีค่าตั้งแต่ -1 ถึง 1 โดยถ้าคะแนนใกล้ 1 หมายความว่ากลุ่มนั้นมีความหนาแน่นและแยกออกจากกลุ่มอื่นได้ดี ค่าใกล้ 0 หมายถึงกลุ่มที่มีการทับซ้อนกัน โดยตัวอย่างอยู่ใกล้กับเส้นแบ่งเขตของกลุ่มข้างเคียง [แหล่งที่มา](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam)

วิธีการเฉลี่ย Silhouette จะคำนวณค่าเฉลี่ยของ Silhouette ของตัวอย่างสำหรับค่าต่าง ๆ ของ *k* คะแนน Silhouette เฉลี่ยที่สูงบ่งบอกถึงการจัดกลุ่มที่ดี

ฟังก์ชัน `silhouette` ในแพ็กเกจ cluster ใช้สำหรับคำนวณค่าเฉลี่ยความกว้างของ Silhouette

> Silhouette สามารถคำนวณได้ด้วย [ระยะทาง](https://en.wikipedia.org/wiki/Distance "Distance") เมตริกใด ๆ เช่น [ระยะทางแบบยุคลิด](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") หรือ [ระยะทางแบบแมนฮัตตัน](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") ซึ่งเราได้พูดถึงใน [บทเรียนก่อนหน้า](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb)


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


คะแนนของเราคือ **.549** ซึ่งอยู่ตรงกลางพอดี นี่บ่งชี้ว่าข้อมูลของเราไม่ได้เหมาะสมกับการจัดกลุ่มประเภทนี้มากนัก ลองมาดูกันว่าเราสามารถยืนยันข้อสันนิษฐานนี้ด้วยการมองเห็นได้หรือไม่ [แพ็กเกจ factoextra](https://rpkgs.datanovia.com/factoextra/index.html) มีฟังก์ชัน (`fviz_cluster()`) สำหรับการแสดงผลการจัดกลุ่มแบบภาพ


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


การที่กลุ่มข้อมูลมีการทับซ้อนกันแสดงให้เห็นว่าข้อมูลของเราอาจไม่เหมาะสมกับการจัดกลุ่มประเภทนี้นัก แต่เรามาลองทำต่อกันเถอะ

## 4. การกำหนดจำนวนกลุ่มที่เหมาะสม

คำถามพื้นฐานที่มักเกิดขึ้นใน K-Means clustering คือ - หากเราไม่มีป้ายกำกับคลาสที่รู้ล่วงหน้า เราจะทราบได้อย่างไรว่าควรแบ่งข้อมูลออกเป็นกี่กลุ่ม?

วิธีหนึ่งที่เราสามารถลองหาคำตอบได้คือการใช้ตัวอย่างข้อมูลเพื่อ `สร้างโมเดลการจัดกลุ่มหลายชุด` โดยเพิ่มจำนวนกลุ่มทีละขั้น (เช่น จาก 1-10) และประเมินเมตริกการจัดกลุ่ม เช่น **Silhouette score**

เรามาลองกำหนดจำนวนกลุ่มที่เหมาะสมโดยการคำนวณอัลกอริธึมการจัดกลุ่มสำหรับค่าต่าง ๆ ของ *k* และประเมิน **Within Cluster Sum of Squares** (WCSS) กันดีกว่า โดย WCSS หรือผลรวมของกำลังสองภายในกลุ่มทั้งหมดนั้นใช้วัดความกระชับของการจัดกลุ่ม ซึ่งเราต้องการให้ค่าต่ำที่สุดเท่าที่จะเป็นไปได้ เพราะค่าที่ต่ำกว่าหมายความว่าจุดข้อมูลอยู่ใกล้กันมากขึ้น

เรามาสำรวจผลกระทบของการเลือกค่า `k` ที่แตกต่างกัน ตั้งแต่ 1 ถึง 10 ต่อการจัดกลุ่มนี้กันเถอะ


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


ตอนนี้เรามีผลรวมของภายในคลัสเตอร์ทั้งหมด (tot.withinss) สำหรับแต่ละอัลกอริทึมการจัดกลุ่มที่มีศูนย์ *k* เราใช้ [วิธีข้อศอก](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) เพื่อหาจำนวนคลัสเตอร์ที่เหมาะสมที่สุด วิธีนี้ประกอบด้วยการวาดกราฟ WCSS เป็นฟังก์ชันของจำนวนคลัสเตอร์ และเลือก [ข้อศอกของกราฟ](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") เป็นจำนวนคลัสเตอร์ที่ใช้


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


กราฟแสดงการลดลงอย่างมากของ WCSS (ซึ่งหมายถึง *ความกระชับ* ที่มากขึ้น) เมื่อจำนวนคลัสเตอร์เพิ่มขึ้นจากหนึ่งเป็นสอง และมีการลดลงที่สังเกตได้ชัดเจนอีกครั้งจากสองเป็นสามคลัสเตอร์ หลังจากนั้น การลดลงจะไม่เด่นชัดเท่าเดิม ทำให้เกิด `elbow` 💪ในกราฟประมาณที่สามคลัสเตอร์ นี่เป็นสัญญาณที่ดีที่บ่งบอกว่ามีคลัสเตอร์ของจุดข้อมูลที่แยกกันได้ดีประมาณสองถึงสามคลัสเตอร์

ตอนนี้เราสามารถดำเนินการและดึงโมเดลการจัดกลุ่มที่ `k = 3` ได้แล้ว:

> `pull()`: ใช้สำหรับดึงคอลัมน์เดียว
>
> `pluck()`: ใช้สำหรับเข้าถึงโครงสร้างข้อมูล เช่น ลิสต์


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


เยี่ยมเลย! มาลองดูการจัดกลุ่มที่ได้กันดีกว่า สนใจเพิ่มความโต้ตอบด้วย `plotly` ไหม?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


บางทีเราอาจคาดหวังว่ากลุ่มแต่ละกลุ่ม (แสดงด้วยสีต่างๆ) จะมีประเภทที่แตกต่างกัน (แสดงด้วยรูปร่างต่างๆ)

ลองมาดูความแม่นยำของโมเดลกัน


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


ความแม่นยำของโมเดลนี้ไม่แย่ แต่ก็ไม่ได้ดีมากนัก อาจเป็นเพราะข้อมูลไม่เหมาะสมกับการใช้ K-Means Clustering ข้อมูลนี้มีความไม่สมดุลกันมาก มีความสัมพันธ์ระหว่างคอลัมน์น้อย และมีความแปรปรวนระหว่างค่าของคอลัมน์สูงเกินไปที่จะจัดกลุ่มได้ดี ในความเป็นจริง กลุ่มที่เกิดขึ้นอาจได้รับอิทธิพลหรือถูกบิดเบือนอย่างมากจากสามหมวดหมู่ของประเภทที่เรากำหนดไว้ข้างต้น

อย่างไรก็ตาม นี่เป็นกระบวนการเรียนรู้ที่น่าสนใจมาก!

ในเอกสารของ Scikit-learn คุณจะเห็นว่าโมเดลแบบนี้ ที่มีการจัดกลุ่มที่ไม่ชัดเจน มีปัญหาเรื่อง 'ความแปรปรวน':

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>อินโฟกราฟิกจาก Scikit-learn</figcaption>



## **ความแปรปรวน**

ความแปรปรวนถูกนิยามว่าเป็น "ค่าเฉลี่ยของผลต่างกำลังสองจากค่าเฉลี่ย" [แหล่งข้อมูล](https://www.mathsisfun.com/data/standard-deviation.html) ในบริบทของปัญหาการจัดกลุ่มนี้ หมายถึงข้อมูลที่ค่าต่างๆ ในชุดข้อมูลมีแนวโน้มที่จะเบี่ยงเบนจากค่าเฉลี่ยมากเกินไป

✅ นี่เป็นช่วงเวลาที่ดีในการคิดถึงวิธีต่างๆ ที่คุณสามารถแก้ไขปัญหานี้ได้ ลองปรับข้อมูลอีกเล็กน้อย? ใช้คอลัมน์ที่แตกต่างกัน? ใช้อัลกอริทึมที่แตกต่างกัน? เคล็ดลับ: ลอง [ปรับขนาดข้อมูลของคุณ](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) เพื่อทำให้ข้อมูลเป็นมาตรฐานและทดสอบคอลัมน์อื่นๆ

> ลองใช้ '[เครื่องคำนวณความแปรปรวน](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' เพื่อทำความเข้าใจแนวคิดนี้เพิ่มเติม

------------------------------------------------------------------------

## **🚀ความท้าทาย**

ใช้เวลาสักครู่กับโน้ตบุ๊กนี้ ปรับพารามิเตอร์ต่างๆ คุณสามารถปรับปรุงความแม่นยำของโมเดลได้โดยการทำความสะอาดข้อมูลเพิ่มเติม (เช่น การลบค่าผิดปกติ)? คุณสามารถใช้น้ำหนักเพื่อให้ความสำคัญกับตัวอย่างข้อมูลบางตัวมากขึ้น คุณสามารถทำอะไรอีกเพื่อสร้างกลุ่มที่ดีกว่า?

เคล็ดลับ: ลองปรับขนาดข้อมูลของคุณ มีโค้ดที่ถูกคอมเมนต์ไว้ในโน้ตบุ๊กที่เพิ่มการปรับขนาดมาตรฐานเพื่อทำให้คอลัมน์ข้อมูลมีลักษณะคล้ายกันมากขึ้นในแง่ของช่วง คุณจะพบว่าแม้คะแนน silhouette จะลดลง แต่ 'จุดหัก' ในกราฟข้อศอกจะเรียบขึ้น นี่เป็นเพราะการปล่อยให้ข้อมูลไม่ได้ปรับขนาดทำให้ข้อมูลที่มีความแปรปรวนน้อยมีน้ำหนักมากขึ้น อ่านเพิ่มเติมเกี่ยวกับปัญหานี้ [ที่นี่](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226)

## [**แบบทดสอบหลังการบรรยาย**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **การทบทวนและการศึกษาด้วยตนเอง**

-   ลองดูตัวจำลอง K-Means [เช่นนี้](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/) คุณสามารถใช้เครื่องมือนี้เพื่อแสดงภาพจุดข้อมูลตัวอย่างและกำหนดจุดศูนย์กลางของมัน คุณสามารถแก้ไขความสุ่มของข้อมูล จำนวนกลุ่ม และจำนวนจุดศูนย์กลาง สิ่งนี้ช่วยให้คุณเข้าใจวิธีการจัดกลุ่มข้อมูลได้หรือไม่?

-   นอกจากนี้ ลองดู [เอกสารประกอบเกี่ยวกับ K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) จาก Stanford

ต้องการลองใช้ทักษะการจัดกลุ่มที่คุณเพิ่งเรียนรู้กับชุดข้อมูลที่เหมาะสมกับ K-Means clustering? โปรดดู:

-   [การฝึกและประเมินโมเดลการจัดกลุ่ม](https://rpubs.com/eR_ic/clustering) โดยใช้ Tidymodels และเพื่อนๆ

-   [การวิเคราะห์กลุ่มด้วย K-Means](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [การจัดกลุ่มด้วย K-Means โดยใช้หลักการข้อมูลที่เป็นระเบียบ](https://www.tidymodels.org/learn/statistics/k-means/)

## **งานที่ได้รับมอบหมาย**

[ลองใช้วิธีการจัดกลุ่มที่แตกต่างกัน](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## ขอขอบคุณ:

[Jen Looper](https://www.twitter.com/jenlooper) สำหรับการสร้างเวอร์ชัน Python ดั้งเดิมของโมดูลนี้ ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) สำหรับการสร้างภาพประกอบที่น่าทึ่งซึ่งทำให้ R น่าสนใจและเข้าถึงได้มากขึ้น ค้นหาภาพประกอบเพิ่มเติมได้ที่ [แกลเลอรีของเธอ](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM)

เรียนรู้อย่างมีความสุข,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>ภาพประกอบโดย @allison_horst</figcaption>



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
